# search
input category, gender, baseColour, season,	usage to get existing image 

In [1]:
import pandas as pd
import copy

In [2]:
train_data = pd.read_csv("data/train.csv")
train_data

,id,category,gender,baseColour,season,usage,noisyTextDescription
0,36274,Scarves,Women,Grey,Summer,Casual,Femella Women Ankle-Length Grey AQ-S800WD-1EVD...
1,15129,Flip Flops,Unisex,Green,Summer,Casual,Converse Unisex Casual Skirts Slipper
2,58976,Topwear,Women,Red,Summer,Ethnic,Velia Women Acetone Kurta
3,32922,Sandal,Men,Brown,Summer,Casual,Enroute Men Leather Brown Sandals
4,29561,Topwear,Women,Pink,Fall,Ethnic,Aneri Exclusive Anu Pink Inspirartion
...,...,...,...,...,...,...,...
21622,5435,Innerwear,Men,Black,Summer,Casual,SHE-3802D-1ADR Men Black Briefs
21623,27880,Belts,Men,Brown,Summer,Casual,Adipower & Taylor Men Brown Quince
21624,38385,Shoes,Women,Beige,Winter,Casual,Rocia Women R348 Flats
21625,34853,Watches,Men,Steel,Winter,Casual,Fastrack Men Black Amethyst Watch


In [6]:
train_data['category'].value_counts()

Topwear                     7689
Shoes                       3607
Bags                        1517
Bottomwear                  1326
Watches                     1248
Innerwear                    910
Eyewear                      531
Jewellery                    526
Fragrance                    521
Sandal                       463
Flip Flops                   461
Wallets                      418
Belts                        382
Socks                        340
Dress                        239
Loungewear and Nightwear     222
Saree                        214
Lips                         193
Nails                        150
Ties                         130
Headwear                     128
Makeup                       118
Accessories                   64
Free Gifts                    61
Apparel Set                   60
Scarves                       59
Cufflinks                     50
Name: category, dtype: int64

In [ ]:
    def pressed(self):
        category = self.comboBox_c.currentText()
        season = self.comboBox_s.currentText()
        gender = self.comboBox_g.currentText()
        print(category, season, gender)
        # self.pushButton.clicked.connect(self.pressed)

In [4]:
train_data['gender'].value_counts()

Men       10880
Women      8995
Unisex     1023
Boys        411
Girls       318
Name: gender, dtype: int64

In [5]:
train_data['season'].value_counts()

Summer    10492
Fall       5643
Winter     4231
Spring     1261
Name: season, dtype: int64

In [11]:
category = "Topwear"
season = "Summer"
gender = "Men"

def filter_data_by_choice(train_data, category, season, gender):
    filtered_data = train_data
    if category != "All":
        filtered_data = filtered_data.loc[filtered_data["category"] == category,]

    if season != "All":
        filtered_data = filtered_data.loc[filtered_data["season"] == season,]

    if gender != "All":
        filtered_data = filtered_data.loc[filtered_data["gender"] == gender,]
    return len(filtered_data),list(filtered_data['id'])

In [13]:
filtered_data = train_data
if category != "All":
    filtered_data = filtered_data.loc[filtered_data["category"] == category,]

if season != "All":
    filtered_data = filtered_data.loc[filtered_data["season"] == season,]

if gender != "All":
    filtered_data = filtered_data.loc[filtered_data["gender"] == gender,]

image_num = len(filtered_data)
image_list = list(filtered_data['id'])

### search by text

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost as xgb
import re
from nltk.corpus import stopwords
import keras.preprocessing.text
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
import keras
# from keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, SpatialDropout1D, Embedding, LSTM
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping

Using TensorFlow backend.


In [2]:
import pickle

with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [3]:
model_lstm = load_model('model/best_model_lstm.h5') 

In [8]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 

    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

testtext = 'Chromozome Men Black Fashion Vest'
cleaned_text = clean_text(testtext)

In [9]:
X = tokenizer.texts_to_sequences([cleaned_text])
X = pad_sequences(X, maxlen=20)
X

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0, 173,   1,   5, 736, 246]])

In [10]:
# first stage predict for test data
test_pred = model_lstm.predict(X)
pred_label = np.argmax(test_pred)

category_mapping = {0: 'Accessories',
 1: 'Apparel Set',
 2: 'Bags',
 3: 'Belts',
 4: 'Bottomwear',
 5: 'Cufflinks',
 6: 'Dress',
 7: 'Eyewear',
 8: 'Flip Flops',
 9: 'Fragrance',
 10: 'Free Gifts',
 11: 'Headwear',
 12: 'Innerwear',
 13: 'Jewellery',
 14: 'Lips',
 15: 'Loungewear and Nightwear',
 16: 'Makeup',
 17: 'Nails',
 18: 'Sandal',
 19: 'Saree',
 20: 'Scarves',
 21: 'Shoes',
 22: 'Socks',
 23: 'Ties',
 24: 'Topwear',
 25: 'Wallets',
 26: 'Watches'}
category_mapping[int(pred_label)]


'Innerwear'